In [ ]:
!pip install scikit-learn==1.0.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.24.0 requires scikit-learn>=1.2.2, but you have scikit-learn 1.0.2 which is incompatible.


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)
bostonDF['PRICE'] = boston.target
bostonDF.head()

### Stochastic Gradient Descent와 Mini Batch Gradient Descent 구현
* SGD 는 전체 데이터에서 **한건만** 임의로 선택하여 Gradient Descent 로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용
* Mini Batch GD는 전체 데이터에서 **Batch 건수만큼** 데이터를 선택하여 Gradient Descent로 Weight/Bias Update 계산한 뒤 Weight/Bias 적용

In [ ]:
# RM, LSTAT 속성만 스케일링
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])

### 반복 시 일정한 batch 크기만큼의 데이터를 random하게 가져와서 GD를 수행하는 Mini-Batch GD 수행

In [ ]:
def get_update_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate=0.01):

    # 데이터 건수
    N = target_batch.shape[0]

    # 예측 값
    predicted_batch = bias + (w1 * rm_batch) + (w2 * lstat_batch)

    # 실제값과 예측값의 차이
    diff_batch = target_batch - predicted_batch

    # bias 를 array 기반으로 구하기 위해서 설정
    bias_factors = np.ones((N, ))

    # weight와 bias를 얼마나 update할 것인지를 계산
    w1_update = (-2/N) * learning_rate * (np.dot(rm_batch.T , diff_batch))
    w2_update = (-2/N) * learning_rate * (np.dot(lstat_batch.T , diff_batch))
    bias_update =  (-2/N) * learning_rate * (np.dot(bias_factors.T , diff_batch))

    # Mean Squared Error값을 계산.-> Loss는 전체 학습 데이터 기반으로 구해야하므로 뒤에서 계산
    # mse_loss = np.mean(np.square(diff))

    # weight와 bias가 update되어야 할 값 반환
    return bias_update, w1_update, w2_update

In [ ]:
# batch_random_gradient_descent()는 인자로 batch_size(배치 크기)를 입력 받음
def batch_random_gradient_descent(features, target, iter_epochs=1000, batch_size=30, verbose=1):

    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.ones((1,))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨
    learning_rate = 0.01
    rm = features[:, 0]
    lstat = features[:, 1]

    # NumPy 난수 생성기의 시드(seed) 값을 2024로 설정 -> 같은 시드를 사용하면 항상 같은 무작위 수가 생성
    np.random.seed(2024)

    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행
    for i in range(iter_epochs):

        # batch_size 갯수만큼 데이터를 임의로 선택 -> np.random.choice 결과 확인
        batch_index = np.random.choice(target.shape[0], batch_size)

        rm_batch = rm[batch_index]
        lstat_batch = lstat[batch_index]
        target_batch = target[batch_index]

        # Batch GD 기반으로 Weight/Bias의 \ 구함
        bias_update, w1_update, w2_update = get_update_value_batch(bias, w1, w2, rm_batch, lstat_batch, target_batch, learning_rate)

        # Batch GD로 구한 weight/bias의 update 적용
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update

        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)

            # Loss는 전체 학습 데이터 기반으로 구해야 함
            predicted = (w1 * rm) + (w2 * lstat) + bias
            diff = target - predicted
            mse_loss = np.mean(np.square(diff))
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', mse_loss)

    return w1, w2, bias

In [ ]:
# batch_random_gradient_descent 함수 호출로 학습(epochs 5000번, batch_size=30)
w1, w2, bias = batch_random_gradient_descent(scaled_features, bostonDF['PRICE'].values, 5000, 30, 1)
print('####### 최종 w1, w2, bias #######')
print(w1, w2, bias)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch: 2502 / 5000
w1: [23.75444477] w2: [-21.32461622] bias: [16.47790331] loss: 30.99333988744427
Epoch: 2503 / 5000
w1: [23.74170602] w2: [-21.3301491] bias: [16.45217487] loss: 31.001098722189386
Epoch: 2504 / 5000
w1: [23.74649275] w2: [-21.33557247] bias: [16.4546586] loss: 30.99819362026142
Epoch: 2505 / 5000
w1: [23.73029993] w2: [-21.34066685] bias: [16.42532264] loss: 31.010341331862765
Epoch: 2506 / 5000
w1: [23.73309028] w2: [-21.33641535] bias: [16.43665648] loss: 31.006728866109366
Epoch: 2507 / 5000
w1: [23.73697202] w2: [-21.32645767] bias: [16.44551984] loss: 31.00476914972433
Epoch: 2508 / 5000
w1: [23.72241832] w2: [-21.32976326] bias: [16.42568395] loss: 31.01445822809709
Epoch: 2509 / 5000
w1: [23.71886747] w2: [-21.33489184] bias: [16.41630138] loss: 31.017953428682123
Epoch: 2510 / 5000
w1: [23.71069425] w2: [-21.34295001] bias: [16.3941457] loss: 31.027931564950368
Epoch: 2511 / 5000
w1: [23.73948566] w2: [-21.34976597] bias: 

In [ ]:
# weight와 bias로 예측 값 생성 후 dataFrame에 'PREDICTED_PRICE_BATCH_RANDOM' 컬럼 추가
predicted = (scaled_features[:, 0] * w1) + (scaled_features[:, 1] * w2) + bias
bostonDF['PREDICTED_PRICE_MGD'] = predicted
bostonDF.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE_MGD
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,29.076399
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.606428
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.694861
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.486336
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.660272
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21,28.7,28.213546
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9,21.450155
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15,27.1,17.854446
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93,16.5,8.164767
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10,18.9,18.365376
